In [1]:
import numpy as np
import random
import re
import time
from threading import Thread, Semaphore

In [4]:
class Puzzle:
    agents = []
    grid = []
    # initialize the board with the size of the puzzle  
    def __init__(self, row_size, fullfilness):
        self.fullfilness = fullfilness/100
        self.row_size = self.col_size = Puzzle.row_size = Puzzle.col_size= row_size
        self.grid_size = self.row_size ** 2
        
        # randomly generate the position of the agents and values
        pos = [(row,col) for row in range(0,self.row_size) for col in range(0,self.col_size)]
        self.max_agents = self.grid_size-1
        self.nbAgents = int(self.max_agents * self.fullfilness)
        
        self.target = random.sample(pos, k=self.nbAgents)
        self.position = random.sample(pos, k=self.nbAgents)
        
        Puzzle.grid = [[None for _ in range(self.col_size)] for _ in range(self.row_size)]
    
    def showGrid():
        print("\n".join([' '.join(list(map(str,x))) for x in Puzzle.grid]))
 
 
               
class Agent(Thread): 
    
    semaphore = Semaphore(1)
    
    def __init__(self, current_postition, target_position):
        Thread.__init__(self)
        self.current_position = current_postition
        self.target_position = target_position
        Puzzle.grid[self.current_position[0]][self.current_position[1]] = self
        Puzzle.agents.append(self)
        self.running = True
        
    def run(self):
        while self.running:
            self.semaphore.acquire()
            self.move_agent()
            self.semaphore.release()
            time.sleep(0.01)
            
    def move_agent(self):
        neighbors = self.get_neighbors()
        
        for neighbor, pos in neighbors:
            if neighbor == None:
                current_position = self.current_position
                self.current_position = pos
                Puzzle.grid[self.current_position[0]][self.current_position[1]] = self
                Puzzle.grid[current_position[0]][current_position[1]] = None  
                return
            

    def get_neighbors(self):
        row= self.current_position[0]
        col= self.current_position[1]
        
        neighbors = []
        if row > 0:
            neighbors.append([Puzzle.grid[row-1][col], (row-1, col)])
        if row < Puzzle.row_size-1:
            neighbors.append([Puzzle.grid[row+1][col], (row+1, col)])
        if col > 0:
            neighbors.append([Puzzle.grid[row][col-1], (row, col-1)])
        if col < Puzzle.col_size-1:
            neighbors.append([Puzzle.grid[row][col+1], (row, col+1)])
        return neighbors      
    

In [5]:
Puzzle.agents

[]

In [7]:
Puzzle.agents = []
Puzzle.grid=[]

p = Puzzle(3, 30)
for target, pos in zip(p.position, p.target):
    Agent(pos, target)
    

for agent in Puzzle.agents:
    agent.start()
    
max_time= 5 # seconds
init_time = time.time()
done = False

while not done:
    time.sleep(1)
    Puzzle.showGrid()
    print("\n")
    
    if time.time() - init_time > max_time:
        done = True
        print("Timeout")

<Agent(Thread-8, started 13239058432)> None <Agent(Thread-5, started 13188689920)>
None <Agent(Thread-6, started 13205479424)> None
<Agent(Thread-7, started 13222268928)> None None


None <Agent(Thread-6, started 13205479424)> None
<Agent(Thread-8, started 13239058432)> None <Agent(Thread-5, started 13188689920)>
None <Agent(Thread-7, started 13222268928)> None


None None None
<Agent(Thread-7, started 13222268928)> <Agent(Thread-6, started 13205479424)> <Agent(Thread-8, started 13239058432)>
None <Agent(Thread-5, started 13188689920)> None


None None <Agent(Thread-5, started 13188689920)>
<Agent(Thread-7, started 13222268928)> <Agent(Thread-8, started 13239058432)> None
None <Agent(Thread-6, started 13205479424)> None


None None <Agent(Thread-6, started 13205479424)>
<Agent(Thread-8, started 13239058432)> <Agent(Thread-7, started 13222268928)> None
<Agent(Thread-5, started 13188689920)> None None


Timeout
